In [17]:
import numpy as np
from collections import deque
import copy
import math
import random
import time


NbClients = 5
clientLocs = []

while len(clientLocs) < NbClients + 1:
    x = random.randint(0, NbClients*10)
    y = random.randint(0, NbClients*10)
    
    # On vérifie si le point (x, y) existe déjà
    if [x, y] not in clientLocs:
        clientLocs.append([x, y])

clientLocs = np.array(clientLocs)

print("Client Locations:\n", clientLocs)

# calcule la matrix des distances entre les clients
def distance_matrix(locs):
    n = locs.shape[0] # Calcul du nombre de clients
    dist_matrix = np.zeros((n, n)) # Création d'une matrix n x n remplie de zéros
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = np.linalg.norm(locs[i] - locs[j]) # Calcul de la distance euclidienne entre les clients i et j
    dist_matrix = np.round(dist_matrix).astype(int) # Arrondi des distances et conversion en entiers

    return dist_matrix

matrix = distance_matrix(clientLocs)
print("Distance Matrix:\n", distance_matrix(clientLocs))

filename = f"instance/{NbClients+1}x{NbClients+1}.csv"  
np.savetxt(filename, matrix, delimiter=",", fmt='%d')


Client Locations:
 [[34  4]
 [50  0]
 [23 13]
 [ 9 24]
 [ 5 47]
 [30  0]]
Distance Matrix:
 [[ 0 16 14 32 52  6]
 [16  0 30 48 65 20]
 [14 30  0 18 38 15]
 [32 48 18  0 23 32]
 [52 65 38 23  0 53]
 [ 6 20 15 32 53  0]]


In [2]:
filename = f"instance/matrix_distances_{NbClients+1}x{NbClients+1}.csv"  
np.savetxt(filename, matrix, delimiter=",", fmt='%d')


In [ ]:
import numpy as np
from collections import deque
import copy
import random
import time
import pandas as pd

# Lecture de la matrice
matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()

depot = 0
nbTrucks = 2

# truckCycles[0] = temps de chaque camion
# truckCycles[1] = cycle de chaque camion
truckCycles = [
    [0] * nbTrucks,
    [[] for _ in range(nbTrucks)]
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    for i in listeClient:
        if matrix[cur][i] > 0 and (poidMinTrajet == 0 or matrix[cur][i] < poidMinTrajet):
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    return [i for i in range(len(matrix)) if matrix[sommet][i] > 0]

def poidCycle():
    return sum(truckCycles[0])

def recherche_tabou_cycle(matrix, start):
    matrix_copy = copy.deepcopy(matrix)
    tabou = deque(maxlen=len(matrix)+5)
    tabou.append(start)

    # Initialisation du dépôt pour tous les camions
    for k in range(nbTrucks):
        truckCycles[1][k] = [start]
        truckCycles[0][k] = 0

    while len(tabou) < len(matrix):
        # Choisir le camion avec le temps minimal
        truckAtMove = truckCycles[0].index(min(truckCycles[0]))
        cur = truckCycles[1][truckAtMove][-1]

        voisins = voisinsClientGraphematrix(matrix_copy, cur)
        candidats = [v for v in voisins if v not in tabou]

        if not candidats:
            break

        voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # Retirer l'arête
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0

        # Mettre à jour le cycle et le temps
        truckCycles[1][truckAtMove].append(voisin)
        truckCycles[0][truckAtMove] += temps
        tabou.append(voisin)

def tabou_multi_start(matrix, nb_lancements=20):
    tempsMeilleurCycle = float('inf')
    goodI = -1

    for i in range(nb_lancements):
        # Réinitialiser les camions à chaque lancement
        global truckCycles
        truckCycles = [
            [0] * nbTrucks,
            [[] for _ in range(nbTrucks)]
        ]

        for i in range(nbTrucks):
            while True:
                # Choisir un client aléatoire différent du dépôt
                firstNeighbor = random.randint(1, len(matrix)-1)

                # Vérifier que ce client n'est pas déjà choisi par un autre camion
                deja_choisi = any(firstNeighbor in cycle for cycle in truckCycles[1])

                if not deja_choisi:
                    # Initialiser le cycle du camion avec le dépôt et le premier client
                    truckCycles[1][i] = [depot, firstNeighbor]
                    # Initialiser le temps du camion pour aller au premier client
                    truckCycles[0][i] = matrix[depot][firstNeighbor]
                    break  # sortir de la boucle while


        recherche_tabou_cycle(matrix, depot)

        total = poidCycle()3333333
        print(f"Lancement {i+1} terminé : Temps du cycle = {total}")
        for i in range(nbTrucks):
            print(f"Premier client du camion {i+1} : {truckCycles[1][i][0]+1}")

        if total < tempsMeilleurCycle:
            tempsMeilleurCycle = total
            goodI = i + 1

    return tempsMeilleurCycle, goodI

# Exécution
start_time = time.time()
tempsMeilleurCycle, goodI = tabou_multi_start(matrix)
execution_time_ms = (time.time() - start_time) * 1000

# Affichage
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in range(nbTrucks):
    print(f"Cycle du camion {i+1} : ", " -> ".join(str(x+1) for x in truckCycles[1][i]))
    print(f"Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


TypeError: recherche_tabou_cycle() missing 1 required positional argument: 'start'

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [ ]:
import numpy as np
from collections import deque
import copy
import math
import random
import time
import pandas as pd

matrix = pd.read_csv("instance/6x6.csv", header=None).to_numpy()


depot = 0

nbTrucks = 2

truckCycles = [
    [0] * nbTrucks,               # temps total de chaque camion
    [[] for _ in range(nbTrucks)] # cycle de chaque camion
]

def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin, poidMinTrajet

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle():
    poids_total = 0
    for i in range(nbTrucks):
        poids_total += truckCycles[0][i]
    return poids_total



def recherche_tabou_cycle(matrix, start):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)



    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)
    
    truckAtMove = 0

    # Le sommet courant (celui où on se trouve actuellement)
    cur = truckCycles[1][truckAtMove][-1] # On commence au dépôt


    # Boucle principale de la recherche tabou
    while len(tabou) < len(matrix) - 1:
        truckAtMove = min(truckCycles[0])

        if truckCycles[0][truckAtMove] == 0:
            truckCycles[1][truckAtMove].append(start)
            voisin = truckCycles[1][truckAtMove]
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, truckCycles[1][truckAtMove][-1]) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin, temps = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        truckCycles[-1] [truckAtMove].append(voisin) # On ajoute ce voisin au cycle
        truckCycles[0][truckAtMove] += temps  # On met à jour le temps du truck    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur =truckCycles[1][truckAtMove][-1]  # On avance au voisin choisi



def tabou_multi_start(matrix):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    """

    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(20):

        global truckCycles # On réinitialise les cycles des camions


        start = depot

        firstNeighbor = 0

        for i in range(nbTrucks):
            firstNeighbor = random.randint(1, len(matrix)-1)
            if firstNeighbor not in truckCycles[1] or firstNeighbor != 0:
                truckCycles[1][i] = random.randint(1, len(matrix)-1)
            else:
                i -= 1

        # On effectue une recherche tabou locale à partir de ce sommet
        recherche_tabou_cycle(matrix, start)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1} terminé : Temps du cycle = {poidCycle()} terminé en {(time.time() - start_time) * 1000}ms")
        

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1
        elif poidCycle() < tempsMeilleurCycle:
            tempsMeilleurCycle = poidCycle()
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))


tempsMeilleurCycle, goodI = tabou_multi_start(matrix)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "  Temps du cycle :", tempsMeilleurCycle)
for i in nbTrucks:
    print(f"Cycle du camion {i+1} : ")
    for j in range(2, len(truckCycles[1][i])):
        s = truckCycles[i][j]
        print(s + 1, "-> ", end='')
    print(f"\n Temps total du camion {i+1} : {truckCycles[0][i]}")

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 6


TypeError: 'int' object is not subscriptable

In [8]:

prob_bouchon = 0.3
facteur = 3



def generer_facteur_bouchon(heure):
    """
    Génère un facteur global de bouchon selon l'heure de la journée.
    - Peu de bouchons la nuit
    - Maximal vers 8h et 17h
    """
    # Heure normalisée sur 24h → sinus pour faire un cycle
    intensite = 0.5 + 0.5 * math.sin((heure - 8) / 24 * 2 * math.pi)
    # Variation entre 1.0 et 3.0 environ
    n = random.uniform(-2,2)
    facteur = n + 2.0 * intensite  
    if facteur <= 0:
        facteur = 1
    return facteur

def cout_effectif(i, j, heure):
    """
    Retourne le coût dynamique entre 2 villes à une heure donnée.
    Complexité O(1)
    """
    base = matrix[i][j]
    if base == 0:
        return 0
    
    # Facteur global du trafic (selon l'heure)
    facteur_temps = generer_facteur_bouchon(heure)
    
    # Petite variation pseudo-aléatoire stable (chaque route a sa propre sensibilité)
    # Cela évite que tout augmente/diminue uniformément
    random.seed(i * 1000 + j)  # stable entre runs
    variation = random.uniform(-0.1, 0.1)  # entre -10% et +10%
    
    cout = base * facteur_temps * (1 + variation)
    return round(cout, 2)

def simulation_journee():
    """
    Simule 24 heures avec bouchons évoluant toutes les 4 heures.
    """
    heures = list(range(0, 25, 4))  # 0h, 4h, 8h, 12h, 16h, 20h, 24h
    for h in heures:
        print(f"\n=== Heure {h}h ===")
        facteur = generer_facteur_bouchon(h)
        print(f"Facteur global de bouchon : {facteur:.2f}")
        
        # Exemple : coût de la route (0 -> 5) évolue dans la journée
        cout_05 = cout_effectif(0, 5, h)
        print(f"Coût entre ville 0 et 5 à {h}h : {cout_05}")


simulation_journee()




=== Heure 0h ===
Facteur global de bouchon : 0.96
Coût entre ville 0 et 5 à 0h : 6.15

=== Heure 4h ===
Facteur global de bouchon : 1.10
Coût entre ville 0 et 5 à 4h : 8.08

=== Heure 8h ===
Facteur global de bouchon : 1.97
Coût entre ville 0 et 5 à 8h : 13.41

=== Heure 12h ===
Facteur global de bouchon : 2.83
Coût entre ville 0 et 5 à 12h : 18.73

=== Heure 16h ===
Facteur global de bouchon : 2.83
Coût entre ville 0 et 5 à 16h : 18.73

=== Heure 20h ===
Facteur global de bouchon : 1.97
Coût entre ville 0 et 5 à 20h : 13.41

=== Heure 24h ===
Facteur global de bouchon : 1.10
Coût entre ville 0 et 5 à 24h : 8.08
